***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h1 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:50px;
               font-style:bold;
               color:white;">
        Part VII
    </h1>
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
       Crusader’s Journey in the World of Images Blog Series
    </h2>
</div>

***
by : JP Fabrero

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Importing Libraries
    </h2>
</div>

***

In [1]:
import os
os.environ['SKIMAGE_DATADIR'] = '/tmp/.skimage_cache'

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import pandas as pd
import glob, re, time

# SkImage
from skimage.io import imread, imshow
from skimage.color import rgb2gray
from skimage.morphology import erosion, dilation, opening, closing
from skimage.measure import label, regionprops_table
from skimage.color import label2rgb

# Classifier Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, make_scorer, precision_score
from tqdm.notebook import trange, tqdm

from pyjanitor import auto_toc
toc = auto_toc()

from pickling import load_pkl, save_pkl

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-asao11dh because the default path (/home/jfabrero/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Preface
    </h2>
</div>

***
Today, we apply ourselves and use all of the concepts discussed so far to train a Machine Learning model. Given images of different leaves, we were tasked to make a pipeline and predict each of the leaves' class.

In [2]:
leaves = 'ABCDE'
fig, ax = plt.subplots(1, 5, figsize=(15, 3.75))

for i, leaf in enumerate(leaves):
    for j, img_path in enumerate(glob.glob(f'./leaves/plant{leaf}*')):
        if j == 0:
            image = rgb2gray(imread(img_path))
            ax[i].imshow(image, cmap='gray')
            ax[i].set_title(f'Plant Leaf {leaf}')
            ax[i].tick_params(     
                which='both',
                bottom=False,
                labelbottom=False,
                left=False,
                labelleft=False
            )
toc.add_fig('Sample Prints of Raw Images',
            width=100)

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Data Collection and Preprocessing
    </h2>
</div>

***
This is where the bulk of our knowledge in image processing is utilized. To clean the image, segment each leaf, and extract features, I leveraged the following methods or concepts:

* **Binarization** - Binarized the images in order to minimize artifacts and clearly establish boundaries between objects and background. For this method, I used a threshold relative to the overall values of the image (average pixel value multiplied by a thresholding factor).

* **Erosion and Dilation** - Applied series of erosion and dilation to clean out remaining artifact after binarization, these are typically miniscule blobs found in the image.

* **Connected Component Segmentation** - After the initial, preprocessing methods, segmentation of leaves were now done using `label` and `regionprops`. After close investigation, there are still some noise remaining the processed images - scan lines and overlaps. These remnants, if handled by further erosion/dilation, will reduced the integrity/quality of the leaves. To mitigate this, the method below was done:

* **Masking** - Masked out labels with sizes significantly lower than the population. The idea being is that if that object is very small compared to the populace, then it is most likely not a leaf.

* **Region Properties** - For feature extraction, we used this method. I identified numerous different features and engineered two additional metrics for robustness.

In [3]:
def erode(image, selem, n=1):
    """Perform erosion `n` times"""
    for _ in range(n):
        image = erosion(image, selem)
    
    return image


def dilate(image, selem, n=1):
    """Perform dilation `n` times"""
    for _ in range(n):
        image = dilation(image, selem)
    
    return image


def get_data(binary_factor=1.1, n=3, thresh=0.2, plot=True):
    """Transform images into tabular data"""
    leaves = 'ABCDE'
    df = pd.DataFrame()
    fig, ax = plt.subplots(1, 5, figsize=(15, 3.75))
    
    for i, leaf in enumerate(leaves):
        for j, img_path in enumerate(glob.glob(f'./leaves/plant{leaf}*')):
            
            # Open image and binarize
            image = rgb2gray(imread(img_path))
            image = (image < image.mean()*binary_factor).astype('int')
            
            # Get rid of noise
            selem = np.zeros((3,3))
            selem[:, 1] = 1
            selem[1, :] = 1

            image = erode(image, selem, n)
            image = dilate(image, selem, n)
            
            # Label and get properties
            label_image = label(image)
            props_image = regionprops_table(
                label_image,
                properties=['area',
                            'area_bbox',
                            'area_convex',
                            'area_filled',
                            'axis_major_length',
                            'axis_minor_length',
                            'eccentricity', 
                            'equivalent_diameter_area',
                            'extent',
                            'feret_diameter_max',
                            'perimeter',
                            'perimeter_crofton',
                            'solidity',
                            'moments_hu',
                            'moments_normalized']
            )
            
            # Save into dataframe and add label
            df_image = pd.DataFrame(props_image)
            df_image = (
                df_image[df_image['area'] > df_image['area'].mean()*thresh]
            )
            df_image['ratio'] = df_image['area'] / df_image['perimeter']
            df_image['leaf'] = leaf
            
            
            df = pd.concat([df, df_image.fillna(0)])
            
            if j == 0:
                ax[i].imshow(image, cmap='gray')
                ax[i].set_title(f'Plant Leaf {leaf}')
                ax[i].tick_params(     
                    which='both',
                    bottom=False,
                    labelbottom=False,
                    left=False,
                    labelleft=False
                )
    toc.add_fig('Sample Prints of Processed Images',
                width=100)
    
    df_view = df.copy()
    df_view.columns = [x.replace('_',' ').title() for x in df_view.columns]
    df_view.iloc[:, :-1] = df_view.iloc[:, :-1].applymap('{:,.2f}'.format)
    
    toc.add_table(df_view,
                  'Leaf Properties from Images',
                  index=False,
                  preview=True)
    
    return df

In [4]:
df = get_data()

***
<div class="header" style="
    padding: 20px;
    background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Model Training
    </h2>
</div>

***
Once we have our table of leaf features, we can now run a grid search and train a different ML models. For reliability, we want the best precision.

In [5]:
# Run Auto ML
def auto_ml(X_trainval, y_trainval, ml_dict,
            X_holdout=None, y_holdout=None, cv=None, scoring=None,
            resampler=None, scaler=None, random_state=None, get_data=False):
    """Runs ML algorithms and returns dataframe of results"""
    # Instantiate 
    result_dict = {}
    model_dict = {}
    dfs_trainval = []
    
    
    for key, value in tqdm(ml_dict.items()): # For Every ML Model Passed
        
        steps = []
        # Add Resampling if Needed
        if resampler is not None:
            steps.append(('resampler', resampler(random_state=random_state)))
        
        # Add Scaler if Needed
        if scaler is not None:
            steps.append(('scaler', scaler()))
        
        steps.extend(value[0])
        pipe = Pipeline(steps)
        
        
        search = GridSearchCV(estimator=pipe,
                              param_grid=value[1],
                              cv=cv,
                              scoring=scoring,
                              return_train_score=True,
                              n_jobs=-1,
                              refit=True)

        search.fit(X_trainval, y_trainval)
        
        df_trainval = pd.DataFrame(
            {
                'Train Scores': search.cv_results_['mean_train_score'],
                'Validation Scores': search.cv_results_['mean_test_score'],
            },
            index = search.cv_results_['params']
        )
        dfs_trainval.append(df_trainval)
        
        best_params = []
        for k, val in search.best_params_.items():
            if bool(re.findall(r'__(alpha|C)', k)):
                val = '10e' + str(round(np.log10(val), 2))
            best_params.append(re.findall(r'__(.*)', k)[0]+'='+str(val))
        best_params = ', '.join(best_params)
        
        best_val = abs(search.best_score_)
        
        best_train = abs(
            search.cv_results_['mean_train_score'][search.best_index_]
        )

        name = value[0][-1][0]
        
        # Feature Importance
        best_feature = 'N/A'
        try:
            indices = list(np.argsort(np.abs(search.best_estimator_[name].coef_)))
            best_feature = ', '.join(X_trainval
                                         .iloc[:, indices[:-4:-1]]
                                         .columns
                                         .tolist())
        except:
            try:
                indices = list(np.argsort(np.mean(np.abs(search
                                                         .best_estimator_[name]
                                                         .coef_),
                                                  axis=0)))
                best_feature = ', '.join(X_trainval
                                         .iloc[:, indices[:-4:-1]]
                                         .columns
                                         .tolist())
            except:
                pass

        try:
            indices = list(np.argsort(np.abs(search
                                             .best_estimator_[name]
                                             .feature_importances_)))
            best_feature = ', '.join(X_trainval
                                     .iloc[:, indices[:-4:-1]]
                                     .columns
                                     .tolist())
        except:
            pass

        run_best_params = {a: [b] for a, b in search.best_params_.items()}
        start = time.time()
        best_score = search.score(X_holdout, y_holdout)
        run_time = time.time() - start

        result_dict[key] = {'Best Parameters': best_params,
                            'Train Score': best_train,
                            'Validation Score': best_val,
                            'Test Score': best_score,
                            'Top Predictor': best_feature,
                            'Run Time': run_time}
        
        model_dict[key] = search.best_estimator_
        
    df_resdata = pd.DataFrame.from_dict(result_dict, orient='index')
    
    if get_data:
        return df_resdata, model_dict, dfs_trainval
    else:
        return df_resdata, model_dict

    
def run_autoML(df):
    """Run AutoML with predefined values and variables"""
    ml_dict = {
        'Decision Tree': (
            [('dtree', DecisionTreeClassifier())],
            {'dtree__max_depth': list(range(1, 26))},
        ),
        'Random Forest': (
            [('rforest', RandomForestClassifier())],
            {'rforest__max_depth': list(range(2, 5))},
        ),
        'Gradient Boosting Method': (
            [('gbm', GradientBoostingClassifier())],
            {'gbm__max_depth': list(range(2, 6)),
             'gbm__learning_rate': [0.01, 0.05, 0.1]}
        ),
        'Light GBM': (
            [('lgbm', LGBMClassifier())],
            {'lgbm__max_depth': list(range(2, 6)),
             'lgbm__learning_rate': [0.01, 0.05, 0.1]}
        ),
    }

    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    X_trainval, X_holdout, y_trainval, y_holdout = train_test_split(
        X, y, test_size=0.2, random_state=143, stratify=y
    )

    cv = 10
    scoring = make_scorer(precision_score,
                          average='weighted',
                          zero_division=0)
    
    df_resdata, model_dict = auto_ml(X_trainval=X_trainval,
                                     y_trainval=y_trainval,
                                     ml_dict=ml_dict,
                                     X_holdout=X_holdout,
                                     y_holdout=y_holdout,
                                     cv=cv,
                                     scoring=scoring,)

    scores = ['Train Score', 'Validation Score', 'Test Score']
    df_resdata[scores] = df_resdata[scores].applymap('{:,.2%}'.format)
    
    df_resdata.rename_axis('Model', inplace=True)
    toc.add_table(df_resdata,
                  'Summary Table of Results of AutoML',
                  preview=False,
                  index=True)
    
    return df_resdata, model_dict

In [6]:
results, models = run_autoML(df)

  0%|          | 0/4 [00:00<?, ?it/s]

***
<div class="header" style="
    padding: 20px;
    background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Final Evaluation
    </h2>
</div>

***


In [7]:
def display_conmat(df, model):
    """Display the confusion matrix using the given model"""
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    labels = y.unique()
    con_mat = confusion_matrix(y, model.predict(X))
    fig, ax = plt.subplots(figsize=(15,8))
    
    sns.heatmap(con_mat,
                annot=True,
                xticklabels=labels,
                yticklabels=labels,
                ax=ax,
                vmax=40,
                vmin=-3,
                cmap='Greys',
                cbar=False,
                fmt='.0f',
                annot_kws={'fontsize': 16},
                square=True)
    ax.set_ylabel('True Labels', fontsize=14)
    ax.set_xlabel('Predicted Labels', fontsize=12)
    toc.add_fig('Confusion Matrix', width=100)

In [8]:
display_conmat(df, models['Gradient Boosting Method'])

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        Author's Notes
    </h2>
</div>

***
At this point, it is evident how finding the right application of the methods we've learned is very crucial to the end results. When the different approached were discussed for this assignment, the general direction or sequence of concepts were very similar how we set the thresholds or other parameters really came in to play. I would argue that one my key differentiator was simply masking out the small connected components which are basically noise. Setting the size threshold relative to image's average leaf size was very effective. Other image processing methods definitely contributed to the preparation of images for labelling but it is important not to get caught up overusing these methods.

***
<div class="header" style="
  padding: 20px;
  background: black;">
    <h2 style="font-family:Copperplate, Sans-serif, Arial;
               font-size:30px;
               font-style:bold;
               color:white;">
        References and Acknowledgement
    </h2>
</div>

***
The concepts discussed and images were derived from our MSDS2023 - IIP Course instructed by Benjur Emmanuel L. Borja. ChatGPT was used for concept reviews and writing prompts.

[1] Mega Swampert -Mega Evolve Art by Tomycase on DeviantArt. (2014, June 12). DeviantArt. https://www.deviantart.com/tomycase/art/Mega-Swampert-Mega-Evolve-Art-460315184